Aqui estão os trem de estudo

In [31]:
import gurobipy as gp
import numpy as np
import pandas as pd
from math import *

In [32]:
df_ori_dest = pd.read_csv('./dados/origem_porto.csv')
df_ori_trans = pd.read_csv('./dados/origem_transbordo.csv')
df_trans_porto = pd.read_csv('./dados/transbordo_porto.csv')

In [35]:
df_supply = pd.read_csv('./dados/supply.csv')
df_cap_port = pd.read_csv('./dados/cap_porto.csv')
df_cap_trans = pd.read_csv('./dados/cap_transbordo.csv')
df_demand = pd.read_csv('./dados/demand.csv')

In [36]:
qnt_orig = df_supply.shape[0]
qnt_trans = df_cap_trans.shape[0]
qnt_port = df_cap_port.shape[0]
qnt_cli = df_demand.shape[0]

In [37]:
df_cap_port.shape[0]

3

In [30]:
N = [i for i in range(qnt_orig)]
M = [i + qnt_orig + qnt_port for i in range(qnt_port)]
T = [i + qnt_orig for i in range(qnt_trans)]
K = [0, 1]
O = [10]

supply = {}
for i in N:
    supply[i] = df_supply['0'][i]

demand = {}
for i in M:
    demand[i] = df_cap_port['0'][i - M[0]]

ways = {}
for i in M:
    aux = []
    for t in T:
        aux.append(t)
    ways[i] = {j for j in aux}

cost = {}
for i in N:
    for j in M:
        cost[i, j, 0, 10] = {df_ori_dest[str(j - M[0])][i]}
for i in N:
    for k in T:
        for j in M:
            print(i, j)
            cost[i, j, k, 10] = {df_ori_trans[str(k - T[0])][i] + df_trans_porto[str(j - M[0])][k - T[0]]}

0 7
0 8
0 9
0 7
0 8
0 9
1 7
1 8
1 9
1 7
1 8
1 9
2 7
2 8
2 9
2 7
2 8
2 9
3 7
3 8
3 9
3 7
3 8
3 9


In [24]:
M

[5]

In [252]:
N = [1, 2, 3, 4]
M = [7, 8, 9]
K = [0, 1]
O = [10]

supply  = {1: 2473300, 2: 2826400, 3: 2048100, 4: 5123300}

demand = {7: 6128000, 8: 16589000, 9: 3168000}


# cost = {(1, 7): 125.1, (1, 8): 304.8, (1, 9): 198.7, (1, 5): 102.2, (1, 6): 269.8,
#         (2, 7): 188.0, (2, 8): 240.4, (2, 9): 191.4, (2, 5): 63.2, (2, 6): 205.7,
#         (3, 7): 131.8, (3, 8): 297.5, (3, 9): 177.1, (3, 5): 98.1, (3, 6): 264.2,
#         (4, 7): 190.9, (4, 8): 228.5, (4, 9): 139.8, (4, 5): 106.9, (4, 6): 206.8,
#         (5, 7): 11.2, (5, 8): 10000, (5, 9): 10000, (6, 8): 13.5, (6, 7): 10000, (6, 9): 10000}


ways = {7: {0, 1}, 8: {0, 1}, 9: {0}}
cost = {
    (1, 7, 0): 125.1, (1, 8, 0): 304.8, (1, 9, 0): 198.7,
    (1, 7, 1): 113.4, (1, 8, 1): 283.3,
    (2, 7, 0): 188.0, (2, 8, 0): 240.4, (2, 9, 0): 191.4,
    (2, 7, 1): 74.4, (2, 8, 1): 219.2,
    (3, 7, 0): 131.8, (3, 8, 0): 297.5, (3, 9, 0): 177.1,
    (3, 7, 1): 109.3, (3, 8, 1): 277.7,
    (4, 7, 0): 190.9, (4, 8, 0): 228.5, (4, 9, 0): 139.8,
    (4, 7, 1): 118.1, (4, 8, 1): 220.3,
}


oferta_total = 0
for i in supply:
    oferta_total += supply[i]
demanda_total = 0
for i in demand:
    demanda_total += demand[i]
print(oferta_total, demanda_total)

12471100 25885000


In [253]:
m = gp.Model("probrema")

In [254]:
X = {}
for i in N:
    for j in M:
        for k in K:
            if not k in ways[j]: continue
            for o in O:
                X[i, j, k, o] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="X_{}_{}_{}_{}".format(i, j, k, o))

# for k in K:
#     for j in M:
#         for o in O:
#             X[k, j, o] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="X_{}_{}_{}".format(k, j, o))

# for i in N:
#     for j in M:
#         for o in O:
#             X[i, j, o] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="X_{}_{}_{}".format(i, j, o))

In [255]:
# m.setObjective(
#     gp.quicksum(X[i, k, o] * cost[i, k] for i in N for k in K for o in O) + \
#     gp.quicksum(X[k, j, o] * cost[k, j] for k in K for j in M for o in O) + \
#     gp.quicksum(X[i, j, o] * cost[i, j] for i in N for j in M for o in O)
#     ,
#     sense=gp.GRB.MINIMIZE)

m.setObjective(
    gp.quicksum((X[i, j, k, o] * cost[i, j, k]) for i in N for j in M for k in K for o in O if k in ways[j]),
    sense=gp.GRB.MINIMIZE
)

In [256]:
for i in N:
    m.addConstr(
        gp.quicksum(X[i, j, k, o] for j in M for k in K for o in O if k in ways[j]) == supply[i]
    )

# for k in K:
#     m.addConstr(
#         gp.quicksum(X[i, k, o] for i in N for o in O) == gp.quicksum(X[k, j, o] for j in M for o in O)
#     )

In [257]:
for j in M:
    for k in K:
        if k not in ways[j]: continue
        m.addConstr(
            gp.quicksum(X[i, j, k, o] for i in N for o in O) <= demand[j]
        )

In [258]:
m.addConstrs(
    gp.quicksum(X[i, j, k, o] for i in N for j in M for o in O if k in ways[j]) >= 0 for o in O
)

# m.addConstrs(
#     gp.quicksum(X[i, k, o] for i in N for k in K) >= 0 for o in O
# )

# m.addConstrs(
#     gp.quicksum(X[k, j, o] for k in K for j in M) >= 0 for o in O
# )

{10: <gurobi.Constr *Awaiting Model Update*>}

In [259]:
# Executa o modelo
m.update()
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10 rows, 20 columns and 48 nonzeros
Model fingerprint: 0x56c368fc
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+01, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+06, 2e+07]
Presolve removed 3 rows and 4 columns
Presolve time: 0.01s
Presolved: 7 rows, 16 columns, 28 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3196754e+09   7.928875e+05   0.000000e+00      0s
       3    1.4331492e+09   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.433149150e+09


In [260]:
m.write("out.sol")